# Getting Started with markovml

This tutorial will walk you through using the markovml package for formally verifying properties of Markov processes with learned parameters. We'll start with a quick example and then dive deeper into the features.

## Setup

First, make sure you have the package in your Python path:

In [1]:
# Add the package to your Python path
import sys
sys.path.append("../")

# Essential imports
import numpy as np
import gurobipy as gp
from markovml.markovml import MarkovReward, MarkovReach, MarkovHitting

## Quick Start Example

Let's start with a simple two-state Markov reward process. Think of this as modeling a system that can be in one of two states (like "healthy" and "sick"), with learned transition probabilities between them.

First we set up a `MarkovReward` object.

In [2]:
# Create a simple two-state Markov reward process
mrp = MarkovReward(n_states=2, n_features=2)

Restricted license - for non-production use only - expires 2026-11-23


Next, we set the initial state distribution and rewards.

In [3]:
# Set rewards for each state (1 for state 0, 0 for state 1)
mrp.set_r([1, 0])

# Set initial state distribution (start in state 0)
mrp.set_pi([1, 0])

Here's where things get interesting: we can add a machine learning model and set the parameters of the Markov reward process to be affine functions of the output of the model. Below we train a logistic regression model on some random data.

In [4]:
from sklearn.linear_model import LogisticRegression
# Create and train a simple classifier
X = np.random.rand(100, 2)  # Random features
y = np.random.randint(0, 2, 100)  # Random labels
clf = LogisticRegression().fit(X, y)

# Add the classifier to the Markov process
mrp.add_ml_model(clf)

# Link classifier output to transition probabilities
# P[0,1] = probability of transitioning from state 0 to 1
mrp.set_P([[1 - mrp.ml_outputs[0][0], mrp.ml_outputs[0][0]],
           [0, 1]])

Observe above we first add the ML model to the problem with `add_ml_model`. When we add an ML model, we can access its outputs from `.ml_outputs`. The sole output of the first added model is `ml_outputs[0][0]`.

We then set the transition probabilities to be an (affine) function of the ML model's output. Observe that in `set_P`, we can pass a list of expressions or constants. Similarly with `set_r` and `set_pi`.


We can now define the feature space. By default is is [-inf, inf] for each feature, so we'll constrain them. Note the use of the function `add_feature_constraint`. If you are adding constraints on the feature space, you must use this function. This way, the program knows how to reconstruct the feature space when it solves the smaller optimization problems during the decomposition phase.

In [5]:
# Add constraints on the feature space
mrp.add_feature_constraint(mrp.features[0] >= 65)
mrp.add_feature_constraint(mrp.features[1] >= 100)

Finally, we can optimize the total reward. Here we will maximize it.

In [6]:
# Optimize to maximize the total reward
mrp.optimize(sense="max")

{'status': 'optimal',
 'objective': 33.33333333333331,
 'values': {'pi': [1.0, 0.0],
  'P': [[1.0, 0.0], [0.0, 1.0]],
  'r': [1.0, 0.0],
  'v': [33.333333333333336, -0.0],
  'features': [65.0, 100.0],
  'ml_outputs': [[0.0]]}}

We get a dictionary of values that tells us the status of the optimization problem, the objective value, the values of the parameters, features, and ML outputs.

And that's it! There are lots of other features and options but this is the basic idea. In the rest of the tutorial, we'll go through each of these steps in more detail.

## Building Markov processes

There are three objects in the `markovml` package: `MarkovReward`, `MarkovReach`, and `MarkovHitting`. They correspond to the problem you are trying to solve: the first one builds a Markov reward process to optimize the total reward, the second one builds a Markov reachability process to find the probability of reaching a set of states, and the third one builds a Markov hitting process to find the expected hitting time to a set of states. They have slightly different interfaces and internal operations, but there is lots of common functionality, so it is easy to switch between them. (In fact, they all inherit from the same base class, `AbstractMarkov`, which provides almost all of the functionality.)



For `MarkovReward`, we need to specify the number of states `n_states` and the number of features `n_features`. Optionally, we can specify the discount factor `discount_factor` (default is 0.97), and optionally pass constants for the parameters which are `pi` (initial state distribution), `P` (transition probabilities), and `r` (rewards).


In [7]:
markov = MarkovReward(n_states=2, n_features=2) # must specify these
markov = MarkovReward(n_states=2, n_features=2, discount_factor=0.99) # can specify discount
markov = MarkovReward(n_states=2, n_features=2, r=[1, 0]) # can specify rewards
markov = MarkovReward(n_states=2, n_features=2, pi=[1, 0]) # can specify initial state distribution


For `MarkovReach`, you must specify the number of states `n_states`, the number of features `n_features`, the number of transient states `n_transient`, and the number of target states `n_target`. Optionally, you may pass constants for the parameters which are `pi` (initial distribution over the transient states), `Q` (transitions between transient states), and `R` (transitions from transient to target states).


In [8]:
markov = MarkovReach(n_states=2, n_features=2, n_transient=1, n_targets=1) # must specify these
markov = MarkovReach(n_states=2, n_features=2, n_transient=1, n_targets=1, pi=[1, 0]) # can specify initial state distribution
markov = MarkovReach(n_states=2, n_features=2, n_transient=1, n_targets=1, Q=[[1, 0], [0, 1]])
markov = MarkovReach(n_states=2, n_features=2, n_transient=1, n_targets=1, R=[[1, 0], [0, 1]])

For `MarkovHitting`, you must specify the number of states `n_states`, the number of features `n_features`, and the number of transient states `n_transient`. Optionally, you may pass constants for the parameters which are `pi` (initial distribution over the transient states) and `Q` (transitions between transient states). You may wonder why the number of target states is not specified -- this is because it is not needed to formulate the problem.

In [9]:
markov = MarkovHitting(n_states=2, n_features=2, n_transient=1) # must specify these
markov = MarkovHitting(n_states=2, n_features=2, n_transient=1, pi=[1, 0]) # can specify initial state distribution
markov = MarkovHitting(n_states=2, n_features=2, n_transient=1, Q=[[1, 0], [0, 1]])

As you can see, all three classes are very similar. The only difference is in the parameters that define them and their objective functions. The objective function is implemented in the respective classes. Each of these three classes also has a `set_*` function that can be used to set the parameters of the problem, depending on the class; e.g., `set_pi`, `set_P`, `set_Q`, `set_r`, `set_R`. Other than that, every function is the same.

In the rest of the tutorial, we will focus on `MarkovReward` since that is the "richest" of the three classes.

## Adding ML models

The next step is to add a machine learning model to the problem. This is done with the `add_ml_model` function. This function takes a pretrained model as input, creates a MILP formulation of the problem, and adds it to the overall optimization problem. Below we will train a couple of ML models so that we can add them to the problem later.

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier

X = np.random.rand(10000, 10)  # 10 features

# regressor for rewards
y_reward = 2 * X[:, 0]**2 + np.exp(X[:, 1]) - np.sin(X[:, 2]*3) + np.random.normal(0, 0.1, 10000)
reward_model = DecisionTreeRegressor(max_depth=3).fit(X, y_reward)

# classifier for transition probabilities
y_trans = (X[:, 3] + X[:, 4] > 1).astype(int)
trans_model = LogisticRegression().fit(X, y_trans)

# regressor for different rewards
y_reward2 = -np.log(X[:, 5] + 0.1) + 2 * X[:, 6]**3 + np.random.normal(0, 0.1, 10000)
reward_model2 = LinearRegression().fit(X, y_reward2)

# classifier for more transition probabilities
y_trans2 = (X[:, 7] - 2*X[:, 8] > 0).astype(int)
trans_model2 = DecisionTreeClassifier(max_depth=3).fit(X, y_trans2)

Now we initialize a Markov reward process with `5` states and `10` features.

In [11]:
mrp = MarkovReward(n_states=5, n_features=10)

Next, we add each ML model to the problem.

In [12]:
mrp.add_ml_model(reward_model)
mrp.add_ml_model(trans_model)
mrp.add_ml_model(reward_model2)
mrp.add_ml_model(trans_model2)


After adding an ML model, the MILP formulation is added to the overall optimization problem that is being built up in the `MarkovReward` object. We need a way to access the ML outputs, so we store them in the `ml_outputs` attribute. It is a list of dicts, where the keys of the dicts are indices of the outputs: e.g., `ml_outputs[0]` is the dict of outputs of the first ML model added, `ml_outputs[1]` is the dict of outputs of the second ML model added, and so on. If a model outputs a single value, we can access it as `ml_outputs[0][0]`, `ml_outputs[1][0]`, etc. If a model outputs multiple values, we can access them as `ml_outputs[0][0]`, `ml_outputs[0][1]`, `ml_outputs[0][2]`, etc, in the order in which they are outputted by the ML model.




In [13]:
mrp.ml_outputs

#type: gurobipy._core.tupledict

[{0: <gurobi.Var ml_outputs_0[0]>},
 {0: <gurobi.Var ml_outputs_1[0]>},
 {0: <gurobi.Var ml_outputs_2[0]>},
 {0: <gurobi.Var ml_outputs_3[0]>}]

Internally, these variables are Gurobi variables, so we can construct expressions using them. In fact the dictionary that stores them is a `tupledict` object, which comes from `gurobipy._core`. This will be useful when we want to link the ML outputs to the parameters of the Markov reward process.

We support numerous ML models: see the documentation for a list. Or, you can obtain the list by printing out the following attribute.

In [14]:
for key in mrp._ml_model_registry:
    print(key)

(<class 'sklearn.linear_model._base.LinearRegression'>, <class 'sklearn.linear_model._ridge.Ridge'>, <class 'sklearn.linear_model._coordinate_descent.Lasso'>)
<class 'sklearn.linear_model._logistic.LogisticRegression'>
<class 'sklearn.tree._classes.DecisionTreeRegressor'>
<class 'sklearn.tree._classes.DecisionTreeClassifier'>
<class 'sklearn.ensemble._forest.RandomForestRegressor'>
<class 'sklearn.ensemble._forest.RandomForestClassifier'>
<class 'sklearn.neural_network._multilayer_perceptron.MLPRegressor'>
<class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>
<class 'markovml.utils.models_ext.DecisionRules'>
<class 'markovml.utils.models_ext.SequentialClassifier'>
<class 'torch.nn.modules.container.Sequential'>


There are numerous scikit-learn models supported as well as PyTorch `nn.Sequential` models. Some care has to be taken here as to whether the model is a classifier or a regressor. For more details, see the tutorial `tutorials/pytorch_models.ipynb`. We also provide a custom model class called `DecisionRules` that can be used to construct "if-then" rules in a natural language-like syntax. For more details on this, see the tutorial `tutorials/decisionrules.ipynb`.


## Setting the parameters

We allow the parameters of the Markov process to be bound via *affine equalities* to the ML outputs. Of course, this also means they can be set to constants. There are three ways to do this. 
1. Pass constants at initialization.
2. Use the `set_to_const` or `set_to_ml_output` functions.
3. Use the `set_*` helper functions.



If passing values at initialization, they must be constants. Why? Because at initialization, the ML models have not yet been added, so we cannot refer to the ML output variables.

In [15]:
# pass constants at initialization
MarkovReward(n_states=2, n_features=2, r=[1, 0])

The `set_to_const` and `set_to_ml_output` functions can be used to set the parameters one at a time. They take a variable and a value as input. If we call `set_to_const`, the value must be a constant. If we call `set_to_ml_output`, the value must be an expression that uses the ML output variables. Let's try this out with our `mrp` object.

In [16]:
# setting to constants
mrp.set_to_const(mrp.r[0], 1)
mrp.set_to_const(mrp.r[1], 0)

# setting to functions of ML outputs
mrp.set_to_ml_output(mrp.r[2], mrp.ml_outputs[0][0])
mrp.set_to_ml_output(mrp.r[3], 2*mrp.ml_outputs[0][0]-1)

Recall that the first ML model we added was a regressor for rewards, so we obtain its output and set it equal to `mrp.r[2]`, and we make `mrp.r[3]` equal to `2*mrp.ml_outputs[0][0]-1`.

Recall the parameters of the Markov reward process are `pi`, `P`, and `r`, and we can call them with specific indices to get the variable. Again, internally, these are Gurobi variables, which enables us to construct expressions using them. See section on "Building Markov processes" for the analogs in the case of `MarkovReach` and `MarkovHitting`.


An important caveat to note is that once a parameter has been set, it cannot be changed, due to how things are constructed internally. We will get the following error now if we try to set `mrp.r[2]` to `1`.

In [17]:
# expecting ValueError
mrp.set_to_const(mrp.r[2], 1)


ValueError: r[2] is already set to a constant or ML output

Lastly, it may get cumbersome to set the parameters one at a time. This is where the `set_*` helper functions come in, which allows us to set all of `pi`, `P`, or `r` at once. We will use it to set the `P` and `r` parameters.

In [18]:
mrp.set_pi([mrp.ml_outputs[1][0],                # pi[0] = p
            1 - mrp.ml_outputs[1][0],            # pi[1] = 1-p
            0, 0, 0])

# Complex transition probabilities
# Set P using mix of ML outputs and constants
# Each row must sum to 1
mrp.set_P([
    # Row 0: ML-based transitions
    [mrp.ml_outputs[1][0],                    # p
     1 - mrp.ml_outputs[1][0] - 0.2,         # 1-p-0.2
     0.1,                                     # fixed
     0.1,                                     # fixed
     0],                                      # fixed

    # Row 1: Different ML model
    [mrp.ml_outputs[3][0],                    # q
     0.3,                                     # fixed
     0.7 - mrp.ml_outputs[3][0],             # 0.7-q
     0,                                       # fixed
     0],                                      # fixed

    # Row 2: Combination of both ML models
    [0,                                       # fixed
     mrp.ml_outputs[1][0],                   # p
     mrp.ml_outputs[3][0],                   # q
     1 - mrp.ml_outputs[1][0] - mrp.ml_outputs[3][0],  # 1-p-q
     0],                                      # fixed

    # Row 3: More ML combinations
    [0,                                       # fixed
     0.2,                                     # fixed
     mrp.ml_outputs[1][0],                   # p
     0.5,                                     # fixed
     0.3 - mrp.ml_outputs[1][0]],            # 0.3-p

    # Row 4: Absorbing state
    [0, 0, 0, 0, 1]                          # fixed
])

## Adding linear inequalities on the parameters

We can add linear inequalities on the parameters of the Markov process. This is done with the `add_constraint` function. 



In [19]:
mrp.add_constraint(mrp.r[0] >= 1)

Of course, you can add arbitrarily complex linear inequalities on the parameters. For example, you can place ordering constraints on the parameters, like `mrp.add_constraint(mrp.r[0] >= mrp.r[1])`. A common such constraint is *increasing failure rate*, which enforces a stochastic ordering on the probabilities. There is a helper function for this: `mrp.add_ifr_inequalities()`.

## Defining the feature space

The next crucial step is to define the feature space, referred to as $\mathcal{X}$ in the paper. Our only assumption is that the feature space is MILP-representable. In order to do this, there are two basic functions: `add_feature_constraint` and `add_feature_aux_variable`. The first function is to add linear inequalities, while the second allows to add additional auxiliary variables to the Gurobi model, which can be binary, integer, or continuous, and with prespecified bounds. These two building blocks allow us to define any arbitrary MILP-representable feature space.



Here's an example of adding some linear inequalities on the feature space.

In [20]:
mrp.add_feature_constraint(mrp.features[0] + mrp.features[1] <= 1.5)
mrp.add_feature_constraint(mrp.features[2] >= mrp.features[3])

Now we can introduce a binary auxiliary variable to the problem.

In [21]:
from gurobipy import GRB
aux = mrp.add_feature_aux_variable(name="aux", vtype=GRB.BINARY)
mrp.add_feature_constraint(mrp.features[4] + mrp.features[5] <= 1 + aux)

When adding an auxiliary variable, all keyword arguments are passed to the `addVar` function of Gurobi, meaning you can specify bounds, types, names, etc. See the Gurobi documentation for more details.

Also, recall that every modeling variable in our class is a Gurobi variable, so we can also modify their properties as in any Gurobi model. If you are familiar with Gurobi, this will be very natural. For example, below we set the bounds of each feature to be [-1, 1].


In [22]:
# Define feature space constraints
for i in range(mrp.n_features):
    mrp.features[i].LB = -1
    mrp.features[i].UB = 1

Using `add_feature_constraint` and `add_feature_aux_variable` is important because then the program knows how to reconstruct the feature space when solving the smaller MILPs for each ML model. You can always add any constraints you wish by directly accessing `.model`, which is the Gurobi model object. But, by using these helper functions, you can ensure that the decomposition step works properly.

## Optimizing!

Finally, we can optimize the problem!

In [23]:
mrp.optimize(sense="max", verbose=True)

ValueError: r[4] needs to be set to a constant or linked to an ML output

Aha! Not so fast! Before optimizing, all sorts of checks happen. Namely, all the parameters `pi`, `P`, and `r` *must* be bound via an affine equality (or to a constant). Here, the program indicates that `r[4]` has not been bound to anything.



In [24]:
mrp.set_to_const(mrp.r[4], 10)

Now let's try optimizing again.

In [25]:
mrp.optimize(sense="max", verbose=True)

Set parameter OutputFlag to value 1
Set parameter Presolve to value 0
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.3.0 23D2057)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
Presolve  0

Optimize a model with 4 rows, 20 columns and 15 nonzeros
Model fingerprint: 0xd2bf7c56
Model has 28 simple general constraints
  28 INDICATOR
Variable types: 11 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
  GenCon rhs range [3e-01, 3e+00]
  GenCon coe range [1e+00, 1e+00]
Variable types: 39 continuous, 9 integer (9 binary)
Found heuristic solution: objective 3.2503723

Root relaxation: unbounded, 0 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incum

{'status': 'optimal',
 'objective': 288.747889242459,
 'values': {'pi': [-0.0, 1.0, -0.0, -0.0, -0.0],
  'P': [[-0.0, 0.8, 0.1, 0.1, 0.0],
   [0.010093727469360329, 0.3, 0.6899062725306396, -0.0, -0.0],
   [-0.0, 0.0, 0.010093727469360329, 0.9899062725306397, -0.0],
   [-0.0, 0.2, -0.0, 0.5, 0.3],
   [-0.0, -0.0, -0.0, -0.0, 1.0]],
  'r': [1.0, 0.0, 3.2503723290363933, 5.500744658072787, 10.0],
  'v': [284.1957210570221,
   288.747889242459,
   301.7587804206267,
   307.80162169147525,
   333.3333333333333],
  'features': [0.8362411260604847,
   0.5964297652244557,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   0.3880178779363621,
   0.12625254690647303,
   1.0],
  'ml_outputs': [[3.2503723290363933],
   [0.0],
   [1.502729593951862],
   [0.010093727469360329]]}}

The problem solves to optimality, and returns a dictionary of the status, objective value, and values of the decision variables. You can control various settings of the optimization with the arguments to the `optimize` function. For example, the sense can be either `min` or `max`, and the `verbose` argument controls whether the Gurobi log is printed. You can also pass additional paramters to the Gurobi solver using the `gurobi_params` argument, which accepts a dictionary. With this you can control things like the time limit, the optimality gap, and so on. See the Gurobi documentation for more details. 

There is also an argument `use_decomp` which controls whether the decomposition and bound propagation scheme from the paper is used. You can disable it if you wish for benchmarking purposes. Otherwise, there is practically no reason to disable it. We can try optimizing without decomposition now. One thing to note is that because the Gurobi model persists, we need to reset it before optimizing again in order to have a fair comparison of the runtime (i.e., the Gurobi model still stores the previous optimal solution).

In [26]:
mrp.model.reset()
mrp.optimize(sense="max", verbose=True, use_decomp=False)

Discarded solution information
Set parameter OutputFlag to value 1
Set parameter Presolve to value 0
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.3.0 23D2057)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
Presolve  0

Optimize a model with 50 rows, 73 columns and 127 nonzeros
Model fingerprint: 0x12733c4a
Model has 5 quadratic objective terms
Model has 5 quadratic constraints
Model has 57 simple general constraints
  57 INDICATOR
Variable types: 55 continuous, 18 integer (18 binary)
Coefficient statistics:
  Matrix range     [8e-04, 2e+01]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e-02, 3e+02]
  RHS range        [1e-01, 2e+01]
  GenCon rhs range [1e-02, 3e+00]
  GenCon coe range [1e+00, 1e+00]

Solving non-convex MIQCP

Variable types: 101 continuous, 42 integer (19 bina

{'status': 'optimal',
 'objective': 288.747889242459,
 'values': {'pi': [-0.0, 1.0, -0.0, -0.0, -0.0],
  'P': [[-0.0, 0.8, 0.1, 0.1, 0.0],
   [0.010093727469360329, 0.3, 0.6899062725306396, -0.0, -0.0],
   [-0.0, 0.0, 0.010093727469360329, 0.9899062725306397, -0.0],
   [-0.0, 0.2, -0.0, 0.5, 0.3],
   [-0.0, -0.0, -0.0, -0.0, 1.0]],
  'r': [1.0, 0.0, 3.2503723290363933, 5.500744658072787, 10.0],
  'v': [284.1957210570221,
   288.747889242459,
   301.7587804206267,
   307.80162169147525,
   333.3333333333333],
  'features': [0.8362411260604847,
   0.5964297652244557,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   0.3880178779363621,
   0.12625254690647303,
   1.0],
  'ml_outputs': [[3.2503723290363933],
   [0.0],
   [1.502729593951862],
   [0.010093727469360329]]}}

As you can see, we get the same objective value!

Instead of optimizing, we can also find a feasible solution, as below. You can pass a lower bound and an upper bound for the objective value, and it will determine if a feasible solution exists within that range.

In [27]:
mrp.model.reset()
mrp.find_feasible(lb=0, ub=1000)

Discarded solution information


{'status': 'feasible',
 'values': {'pi': [0.0, 1.0, 0.0, 0.0, 0.0],
  'P': [[0.0, 0.8, 0.1, 0.1, 0.0],
   [0.16236162361623419, 0.3, 0.5376383763837648, 0.0, 0.0],
   [0.0, 0.0, 0.1623616236162343, 0.8376383763837649, 0.0],
   [0.0, 0.2, 0.0, 0.5, 0.3],
   [0.0, 0.0, 0.0, 0.0, 1.0]],
  'r': [1.0, 0.0, 3.2503723290363933, 5.500744658072787, 10.0],
  'v': [277.14710364141985,
   280.554878054622,
   297.7230103596665,
   304.71532233139675,
   333.3333333333333],
  'features': [0.8362411260604847,
   0.5964297652244557,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   1.0,
   0.43699599802494227,
   1.0],
  'ml_outputs': [[3.2503723290363933],
   [0.0],
   [1.4932539761713508],
   [0.16236162361623552]]}}

And again, this time with a tighter bound.

In [28]:
mrp.model.reset()
mrp.find_feasible(lb=0, ub=10)

{'status': 'infeasible', 'message': 'Problem is infeasible'}

Now we get that the problem is infeasible! By the way, if you wish to keep one of the ends "open", i.e., $\pm \infty$, you can pass `None` for the bound. But you have to specify at least one bound.

And that's it! You now understand the full workflow of the `markovml` package. You are ready to start using it. For more details, you can browse the documentation or the other tutorials.